# part0: imports

In [ ]:
%matplotlib inline


import os, sys, pathlib
from pprint import pprint
from tqdm import tqdm
import pickle
from importlib import reload
import logging, warnings
logging.basicConfig(level=logging.ERROR)
warnings.simplefilter("ignore")

import PyPDF2 as ppdf
import pandas as pd
import numpy as np
import sklearn
from sklearn.decomposition import PCA
import scipy.linalg as linalg
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, r2_score
from scipy.stats import wilcoxon, mannwhitneyu


import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator, FormatStrFormatter
from matplotlib.collections import LineCollection

import torch

import pyaldata as pyal

if "__file__" not in dir():
    try:
        NBPath = pathlib.Path.cwd()
        RepoPath = NBPath.parent
        os.chdir(RepoPath)

        from tools import utilityTools as utility
        from tools import dataTools as dt
        from tools import lstm
        import params
        reload(params)
        monkey_defs = params.monkey_defs
        mouse_defs = params.mouse_defs

        set_rc =  params.set_rc_params
        set_rc()
        root = params.root

        os.chdir(RepoPath / 'monkey')
        %run "_dataset-selection.ipynb"

    finally:
        os.chdir(NBPath)

    %run "fig3.ipynb"

    print('Done')

# Part 1

functions that plot each panel.


---

monkey population CCA plot

In [ ]:
@utility.report
def get_full_monkey_data():
    full_list_MCx = []
    for animal, sessionList in GoodDataList[monkey_defs.areas[2]].items():
        if 'Mr' in animal:
            continue  # to remove MrT
        full_list_MCx.append((animal,sessionList))
    full_list_MCx = [(animal,session) for animal,sessions in full_list_MCx for session in set(sessions)]
    # load the DFs
    allDFs_MCx = []
    for animal, session in full_list_MCx:
        path = root/animal/session
        allDFs_MCx.append(monkey_defs.prep_general(dt.load_pyal_data(path)))

    return full_list_MCx, allDFs_MCx

In [ ]:
@utility.report
def plot_monkey_cca_prep(ax, ax_hist, full_list_MCx, allDFs_MCx):
    defs = monkey_defs
    
    pairFileList1 = []
    for I, (animal1,session1) in enumerate(full_list_MCx):
        for J, (animal2,session2) in enumerate(full_list_MCx):
            if J<=I or animal1 == animal2: continue  # to repetitions
            if 'Chewie' in animal1 and 'Chewie' in animal2: continue 
            pairFileList1.append((I,J))

    side1df = [allDFs_MCx[i] for i,_ in pairFileList1]
    side2df = [allDFs_MCx[j] for _,j in pairFileList1]
    AllData1 = dt.get_data_array(side1df, monkey_defs.prep_epoch, area=monkey_defs.areas[2], model=monkey_defs.n_components)
    AllData2 = dt.get_data_array(side2df, monkey_defs.prep_epoch, area=monkey_defs.areas[2], model=monkey_defs.n_components)
    _,_, min_trials, min_time,_ = np.min((AllData1.shape,AllData2.shape),axis=0)
    allCCs=[]
    for sessionData1,sessionData2 in zip(AllData1,AllData2):
        data1 = np.reshape(sessionData1[:,:min_trials,:min_time,:], (-1,defs.n_components))
        data2 = np.reshape(sessionData2[:,:min_trials,:min_time,:], (-1,defs.n_components))
        allCCs.append(dt.canoncorr(data1, data2))
    allCCs = np.array(allCCs).T

    # lower bound
    len_trial = int(np.round(np.diff(defs.WINDOW_exec)/defs.BIN_SIZE))
    
    single_FileList1 = []
    for I, (animal1,session1) in enumerate(full_list_MCx):
        for J, (animal2,session2) in enumerate(full_list_MCx):
            if J<=I or animal1 == animal2: continue  # to repetitions
            if 'Chewie' in animal1 and 'Chewie' in animal2: continue 
            single_FileList1.append((I,J))
    n_iter = params.n_iter * 10
    side1df = [allDFs_MCx[i] for i,_ in single_FileList1]
    side2df = [allDFs_MCx[j] for _,j in single_FileList1]
    AllData1 = defs._get_data_array(side1df, epoch_L=len_trial, area=defs.areas[2], model=defs.n_components)
    AllData1_ = defs._get_data_array(side2df, epoch_L=len_trial, area=defs.areas[2], model=defs.n_components)
    _,_, min_trials, min_time,_ = np.min((AllData1.shape,AllData1_.shape),axis=0)

    CCsL=[]
    for sessionData1,sessionData2 in zip(AllData1,AllData1_):
        r = []
        for n in range(n_iter):
            sessionData1_sh = params.rng.permutation(sessionData1,axis=0)
            sessionData2_sh = params.rng.permutation(sessionData2,axis=0)

            data1 = np.reshape(sessionData1_sh[:,:min_trials,:min_time,:], (-1,defs.n_components))
            data2 = np.reshape(sessionData2_sh[:,:min_trials,:min_time,:], (-1,defs.n_components))
            r.append(dt.canoncorr(data1, data2))
        CCsL.append(r)
    CCsL = np.array(CCsL)
    CCsL = np.percentile(CCsL, 1, axis=1).T

    # Upper bound
    single_FileList1 = []
    for I, _ in enumerate(full_list_MCx):
        single_FileList1.append(I)
    side1df = [allDFs_MCx[i] for i in single_FileList1]
    AllData1 = dt.get_data_array(side1df, monkey_defs.prep_epoch, area=monkey_defs.areas[2], model=monkey_defs.n_components)
    n_shared_trial1 = AllData1.shape[2]
    trialList1 = np.arange(n_shared_trial1)
    CCsU=[]
    for session, sessionData in enumerate(AllData1):
        r = []
        for n in range(n_iter):
            params.rng.shuffle(trialList1)
            # non-overlapping randomised trials
            trial1 = trialList1[:n_shared_trial1//2]
            trial2 = trialList1[-(n_shared_trial1//2):]
            data1 = np.reshape(sessionData[:,trial1,:,:], (-1,defs.n_components))
            data2 = np.reshape(sessionData[:,trial2,:,:], (-1,defs.n_components))
            r.append(dt.canoncorr(data1, data2))
        CCsU.append(r)
    CCsU = np.array(CCsU)
    CCsU = np.percentile(CCsU, 99, axis=1).T

    # plotting
    x_ = np.arange(1,monkey_defs.n_components+1)
    utility.shaded_errorbar(ax, x_, allCCs, color=params.colors.MainCC, marker = 'o')
    utility.shaded_errorbar(ax, x_, CCsU, color=params.colors.UpperCC, marker = '<', ls='--')
    utility.shaded_errorbar(ax, x_, CCsL, color=params.colors.LowerCC, marker = '>', ls=':')

    ax.set_ylim([-.05,1])
    ax.set_xlim([.6,monkey_defs.n_components+.6])
    ax.set_xlabel('Neural mode')
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_ylabel('Canonical correlation')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_bounds([1,monkey_defs.n_components])
    ax.spines['left'].set_bounds([0,1])
    
    #plot the hist
    bins = np.arange(0,1,0.05)
    ax_hist.xaxis.set_visible(False)
    ax_hist.set_facecolor('None')
    ax_hist.spines['bottom'].set_visible(False)
    ax_hist.spines['right'].set_visible(False)
    ax_hist.spines['top'].set_visible(False)
    ax_hist.spines['left'].set_bounds([0,1])
    ax_hist.set_ylim([-.05,1])
    ax_hist.hist(allCCs[:4,:].mean(axis=0), bins=bins, density=True, label=f'Across $(n={allCCs.shape[1]}$)',
            color=params.colors.MainCC, alpha=.8, orientation='horizontal')
    ax_hist.hist(CCsU[:4,:].mean(axis=0), bins=bins, density=True,label=f'Within ($n={CCsU.shape[1]}$)',
            color=params.colors.UpperCC, alpha=.8, orientation='horizontal')
    ax_hist.hist(CCsL[:4,:].mean(axis=0), bins=bins, density=True, label=f'Control ($n={CCsL.shape[1]}$)',
            color=params.colors.LowerCC, alpha=.8, orientation='horizontal')
    
    ax_hist.tick_params('y', direction='out')
    ax_hist.set_yticklabels([])
    ax_hist.legend(loc=(0,-.05))

    #stats ###########################################################################
    allCCs_median = np.median(allCCs[:4,:].mean(axis=0))
    CCsU_median = np.median(CCsU[:4,:].mean(axis=0))
    CCsL_median = np.median(CCsL[:4,:].mean(axis=0))
    allCCs_mean = allCCs[:4,:].mean(axis=0)
    CCsU_mean = CCsU[:4,:].mean(axis=0)
    CCsL_mean = CCsL[:4,:].mean(axis=0)

    #calc stats
    ##for paired stats
    side1CCsU_mean = [CCsU_mean[i] for i,_ in pairFileList1]
    side2CCsU_mean = [CCsU_mean[j] for _,j in pairFileList1]
    allCCsU_mean = side1CCsU_mean + side2CCsU_mean
        
    compare_upper_stats = wilcoxon(np.tile(allCCs_mean,2), allCCsU_mean)
    compare_lower_stats = wilcoxon(allCCs_mean, CCsL_mean)

    ##for unpaired stats
    # compare_upper_stats = mannwhitneyu(allCCs_mean, CCsU_mean)
    # compare_lower_stats = mannwhitneyu(allCCs_mean, CCsL_mean)

    print("Across vs within:", compare_upper_stats)
    print("Across vs control:", compare_lower_stats)

    #annotate stats
    xmin, xmax = ax_hist.get_xlim()
    markerx = xmax+(xmax-xmin)*0.05
    linex = xmax+(xmax-xmin)*0.15
    textx = xmax+(xmax-xmin)*0.25
    line_kwargs = dict(linewidth = 0.5, color = 'k')
    text_kwargs = dict(ha='left', va='center')

    ax_hist.scatter(markerx, allCCs_median, color = params.colors.MainCC, marker = '<')
    ax_hist.scatter(markerx, CCsU_median, color = params.colors.UpperCC, marker = '<')
    ax_hist.scatter(markerx, CCsL_median, color = params.colors.LowerCC, marker = '<')

    ax_hist.plot([linex, linex], [allCCs_median, CCsU_median], **line_kwargs)
    ax_hist.plot([linex, linex], [allCCs_median, CCsL_median], linestyle = '--', **line_kwargs)
    
    # print(dt.get_signif_annot(compare_upper_stats[1]))
    ax_hist.text(textx, (allCCs_median + CCsU_median)/2, dt.get_signif_annot(compare_upper_stats[1]), **text_kwargs)
    ax_hist.text(textx, (allCCs_median + CCsL_median)/2, dt.get_signif_annot(compare_lower_stats[1]), **text_kwargs)

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc()
    fig=plt.figure(dpi=100)
    gs = fig.add_gridspec(nrows=1, ncols=2, width_ratios=(5,1), wspace=0)

    ax = fig.add_subplot(gs[0])
    ax_ = fig.add_subplot(gs[1])

    
    full_list_MCx, allDFs_MCx = get_full_monkey_data()
    
    plot_monkey_cca_prep(ax,ax_, full_list_MCx[10:13], allDFs_MCx)
    
    plt.show()

---

decoding for monkeys

In [ ]:
@utility.report
def plot_monkey_target_decoding(ax, allDFs):
    defs = monkey_defs
    classifier_model = GaussianNB

    within_score = {}
    aligned_score = {}
    unaligned_score = {}
    for i, df1 in enumerate(tqdm(allDFs)):
        if 'J' in df1.monkey[0]:
            continue  # remove Jaco from this analysis
        AllData = dt.get_data_array([df1], defs.prep_epoch, area=defs.areas[2], model=defs.n_components)
        # adding history
        AllData = dt.add_history_to_data_array(AllData,defs.MAX_HISTORY)

        AllData1 = AllData[0,...]
        _,n_trial,n_time,n_comp = AllData1.shape
        # resizing
        X1 = AllData1.reshape((-1,n_comp*n_time))
        AllTar = np.repeat(np.arange(defs.n_targets),n_trial)
        # train the decoder
        _score=cross_val_score(classifier_model(),X1,AllTar,scoring='accuracy', cv=10).mean()
        within_score[df1.session[0]] = _score

        aligned_score[df1.session[0]] = {}
        unaligned_score[df1.session[0]] = {}
        for j, df2 in enumerate(tqdm(allDFs)):
            if j <= i: continue
            if df1.monkey[0] == df2.monkey[0]: continue
            if 'Chewie' in df1.monkey[0] and 'Chewie' in df2.monkey[0]: continue
            if 'J' in df1.monkey[0] or 'J' in df2.monkey[0]: continue  # remove Jaco from this analysis

            AllData = dt.get_data_array([df1,df2], defs.prep_epoch, area=defs.areas[2], model=defs.n_components)

            # adding history
            AllData = dt.add_history_to_data_array(AllData, defs.MAX_HISTORY)

            AllData1 = AllData[0,...] 
            AllData2 = AllData[1,...]
            _,n_trial,n_time,n_comp = AllData1.shape

            # resizing
            X1 = AllData1.reshape((-1,n_comp))
            X2 = AllData2.reshape((-1,n_comp))

            *_,U,V = dt.canoncorr(X1, X2, fullReturn=True)
            U = U.reshape((-1,n_comp*n_time))
            V = V.reshape((-1,n_comp*n_time))
            AllTar = np.repeat(np.arange(defs.n_targets),n_trial)
            trial_index = np.arange(len(AllTar))
            params.rng.shuffle(trial_index)
            X_train, Y_train = U[trial_index,:], AllTar[trial_index]
            params.rng.shuffle(trial_index)
            X_test, Y_test   = V[trial_index,:], AllTar[trial_index]

            # train the decoder
            classifier = classifier_model()
            classifier.fit(X_train, Y_train)
            # test the decoder
            _score = classifier.score(X_test,Y_test)
            aligned_score[df1.session[0]][df2.session[0]]=_score

            #Unaligned

            X1 = X1.reshape((-1,n_comp*n_time))
            X2 = X2.reshape((-1,n_comp*n_time))
            AllTar = np.repeat(np.arange(defs.n_targets),n_trial)
            trial_index = np.arange(len(AllTar))
            params.rng.shuffle(trial_index)
            X_train, Y_train = X1[trial_index,:], AllTar[trial_index]
            params.rng.shuffle(trial_index)
            X_test, Y_test   = X2[trial_index,:], AllTar[trial_index]
            # train the decoder
            classifier = classifier_model()
            classifier.fit(X_train, Y_train)
            # test the decoder
            _score = classifier.score(X_test,Y_test)
            unaligned_score[df1.session[0]][df2.session[0]]=_score

    #======================== PLOTTING

    pop_within = np.array(list(within_score.values()))
    pop_aligned = np.array([val for key in aligned_score for val in aligned_score[key].values()])
    pop_unaligned = np.array([val for key in unaligned_score for val in unaligned_score[key].values()])

    ax.errorbar(1, pop_aligned.mean(), np.std(pop_aligned), label='Across\n' r'(\textit{aligned})',
                color=params.colors.MainCC, fmt='-o', capsize=1.5)    
    ax.errorbar(0, pop_unaligned.mean(), np.std(pop_unaligned), label='Across\n' r'(\textit{unaligned})',
                color=params.colors.LowerCC, fmt='-o', capsize=1.5)
    ax.errorbar(2, pop_within.mean(), np.std(pop_within), label='Within',
                color=params.colors.UpperCC, fmt='-o', capsize=1.5)

    unal_vals = []
    al_vals = []
    wi_vals = []
    for file1, nested_dict in aligned_score.items():
        wi_val1 = within_score[file1]
        for file2, al_val in nested_dict.items():
            wi_val2 = within_score[file2]
            unal_val = unaligned_score[file1][file2]
            ax.plot([0,1,2], [unal_val, al_val, wi_val1],
                    color='gray', lw=.2, zorder=6, marker = 'o', ms=.1, alpha=.2)
            ax.plot([1,2], [al_val, wi_val2],
                    color='gray', lw=.2, zorder=6, marker = 'o', ms=.1, alpha=.2)

            #for stats
            unal_vals.append(unal_val)
            al_vals.append(al_val)
            wi_vals.append(wi_val1)
            wi_vals.append(wi_val2)

    ax.set_xlim([-0.2,2.2])
    ax.set_xticks([0,1,2])
    ax.set_xticklabels(['Across\n' r'(\textit{unaligned})',
                        'Across\n' r'(\textit{aligned})',
                        'Within'])
    ax.set_ylabel('Classification accuracy')
    ax.set_ylim([-.05,1])
    # ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_bounds([0,2])
    ax.spines['left'].set_bounds([0,1])

    #stats ###########################################################################
    #calc stats
    ##for paired stats
    compare_upper_stats = wilcoxon(np.repeat(al_vals,2), wi_vals)
    compare_lower_stats = wilcoxon(al_vals, unal_vals)

    print("Across vs within:", compare_upper_stats)
    print("Across vs control:", compare_lower_stats)

    #annotate stats
    ymin, ymax = ax.get_ylim()
    liney = ymax*0.95
    texty = ymax*1
    line_kwargs = dict(linewidth = 0.5, color = 'k')
    text_kwargs = dict(ha='center', va='center')

    ax.plot([0,1], [liney, liney], **line_kwargs)
    ax.plot([1,2], [liney, liney], linestyle = '--', **line_kwargs)
    
    ax.text(0.5, texty, dt.get_signif_annot(compare_upper_stats[1]), **text_kwargs)
    ax.text(1.5, texty, dt.get_signif_annot(compare_lower_stats[1]), **text_kwargs)



In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc()
    fig=plt.figure(dpi=100)
    ax = fig.add_subplot()

    
    full_list_MCx, allDFs_MCx = get_full_monkey_data()
    
    plot_monkey_target_decoding(ax, allDFs_MCx[10:15])

    plt.show()

---

Mouse CCA plot

In [ ]:
@utility.report
def get_full_mouse_data():
    defs = mouse_defs
    
    animalList = ['mouse-data']
    animalFiles = []
    for animal in animalList:
        animalFiles.extend(utility.find_file(root / animal, 'mat'))

    AllDFs=[]
    for fname in animalFiles:
        df = dt.load_pyal_data(fname)
        df['mouse'] = fname.split(os.sep)[-1][fname.split(os.sep)[-1].find('WR'):].split('_')[0]
        df['file'] = fname.split(os.sep)[-1]
        df = defs.prep_general_mouse(df)
        AllDFs.append(df)

    allDFs_M1 = []
    for df in AllDFs:
        if 'M1_rates' in df.columns:
            allDFs_M1.append(df)


    allDFs_Str = []
    for df in AllDFs:
        if 'Str_rates' in df.columns:
            allDFs_Str.append(df)
            
    return allDFs_M1, allDFs_Str

In [ ]:
@utility.report
def plot_mouse_str_cca(ax, ax_hist, allDFs_Str):
    defs = mouse_defs

    pairFileList2 = []
    for I, df1 in enumerate(allDFs_Str):
        for J, df2 in enumerate(allDFs_Str):
            if J<=I or df1.mouse[0] == df2.mouse[0]: continue  # repetitions
            pairFileList2.append((I,J))

    side1df = [allDFs_Str[i] for i,_ in pairFileList2]
    side2df = [allDFs_Str[j] for _,j in pairFileList2]
    AllData1_ = dt.get_data_array(side1df, defs.exec_epoch, area=defs.areas[1], model=defs.n_components)
    AllData2_ = dt.get_data_array(side2df, defs.exec_epoch, area=defs.areas[1], model=defs.n_components)
    _,_, min_trials_, min_time_,_ = np.min((AllData1_.shape,AllData2_.shape),axis=0)
    allCCs=[]
    for sessionData1,sessionData2 in zip(AllData1_,AllData2_):
        data1 = np.reshape(sessionData1[:,:min_trials_,:min_time_,:], (-1,defs.n_components))
        data2 = np.reshape(sessionData2[:,:min_trials_,:min_time_,:], (-1,defs.n_components))
        allCCs.append(dt.canoncorr(data1, data2))
    allCCs = np.array(allCCs).T

    # upper bound
    AllData2 = dt.get_data_array(allDFs_Str, defs.exec_epoch, area=defs.areas[1], model=defs.n_components)
    n_shared_trial2 = AllData2.shape[2]
    trialList2 = np.arange(n_shared_trial2)
    n_iter = params.n_iter * 10
    CCsU=[]
    for sessionData in AllData2:
        r = []
        for n in range(n_iter):
            params.rng.shuffle(trialList2)
            # non-overlapping randomised trials
            trial1 = trialList2[:n_shared_trial2//2]
            trial2 = trialList2[-(n_shared_trial2//2):]
            data1 = np.reshape(sessionData[:,trial1,:,:], (-1,defs.n_components))
            data2 = np.reshape(sessionData[:,trial2,:,:], (-1,defs.n_components))
            r.append(dt.canoncorr(data1, data2))
        CCsU.append(r)
    CCsU = np.array(CCsU)
    CCsU = np.percentile(CCsU, 99, axis=1).T

    # lower bound
    len_trial = int(np.round(np.diff(defs.WINDOW_exec)/defs.BIN_SIZE))
    pairFileList2 = []
    for I, df1 in enumerate(allDFs_Str):
        for J, df2 in enumerate(allDFs_Str):
            if J<=I or df1.mouse[0] == df2.mouse[0]: continue  # repetitions
            pairFileList2.append((I,J))
    side1df = [allDFs_Str[i] for i,_ in pairFileList2]
    side2df = [allDFs_Str[j] for _,j in pairFileList2]
    AllData1_ = dt.get_data_array(side1df, area=defs.areas[1], model=defs.n_components)
    AllData2_ = dt.get_data_array(side2df, area=defs.areas[1], model=defs.n_components)
    _,_, min_trials_, min_time_,_ = np.min((AllData1_.shape,AllData2_.shape),axis=0)

    CCsL=[]
    for sessionData1,sessionData2 in zip(AllData1_,AllData2_):
        r = []
        for n in range(n_iter):
            sessionData1_sh = params.rng.permutation(sessionData1,axis=0)
            sessionData2_sh = params.rng.permutation(sessionData2,axis=0)
            time_idx = params.rng.integers(min_time_-len_trial)

            data1 = np.reshape(sessionData1_sh[:,:min_trials_,time_idx:time_idx+len_trial,:], (-1,defs.n_components))
            data2 = np.reshape(sessionData2_sh[:,:min_trials_,time_idx:time_idx+len_trial,:], (-1,defs.n_components))
            r.append(dt.canoncorr(data1, data2))
        CCsL.append(r)
    CCsL = np.array(CCsL)
    CCsL = np.percentile(CCsL, 1, axis=1).T


    #====================================PLOTTING
    
    x_ = np.arange(1,defs.n_components+1)
    utility.shaded_errorbar(ax, x_, allCCs, color=params.colors.MainCC, marker = 'o')
    utility.shaded_errorbar(ax, x_, CCsU, color=params.colors.UpperCC, marker = '<', ls='--')
    utility.shaded_errorbar(ax, x_, CCsL, color=params.colors.LowerCC, marker = '>', ls=':')

    ax.set_ylim([-.05,1])
    ax.set_xlim([.6,defs.n_components+.6])
    ax.set_xlabel('Neural mode')
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_ylabel('Canonical correlation')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_bounds([1,defs.n_components])
    ax.spines['left'].set_bounds([0,1])
    
    #plot the hist

    bins = np.arange(0,1,0.05)
    ax_hist.xaxis.set_visible(False)
    ax_hist.set_facecolor('None')
    ax_hist.spines['bottom'].set_visible(False)
    ax_hist.spines['right'].set_visible(False)
    ax_hist.spines['top'].set_visible(False)
    ax_hist.spines['left'].set_bounds([0,1])
    ax_hist.set_ylim([-.05,1])
    ax_hist.hist(allCCs[:4,:].mean(axis=0), bins=bins, density=True, label=f'($n={allCCs.shape[1]}$)',
            color=params.colors.MainCC, alpha=.8, orientation='horizontal')
    ax_hist.hist(CCsU[:4,:].mean(axis=0), bins=bins, density=True,label=f'($n={CCsU.shape[1]}$)',
            color=params.colors.UpperCC, alpha=.8, orientation='horizontal')
    ax_hist.hist(CCsL[:4,:].mean(axis=0), bins=bins, density=True, label=f'($n={CCsL.shape[1]}$)',
            color=params.colors.LowerCC, alpha=.8, orientation='horizontal')
    
    ax_hist.tick_params('y', direction='out')
    ax_hist.set_yticklabels([])
    ax_hist.legend(loc=(0,-.05))

    #stats ###########################################################################
    allCCs_median = np.median(allCCs[:4,:].mean(axis=0))
    CCsU_median = np.median(CCsU[:4,:].mean(axis=0))
    CCsL_median = np.median(CCsL[:4,:].mean(axis=0))
    allCCs_mean = allCCs[:4,:].mean(axis=0)
    CCsU_mean = CCsU[:4,:].mean(axis=0)
    CCsL_mean = CCsL[:4,:].mean(axis=0)

    #calc stats
    ##for paired stats
    side1CCsU_mean = [CCsU_mean[i] for i,_ in pairFileList2]
    side2CCsU_mean = [CCsU_mean[j] for _,j in pairFileList2]
    allCCsU_mean = side1CCsU_mean + side2CCsU_mean
        
    compare_upper_stats = wilcoxon(np.tile(allCCs_mean,2), allCCsU_mean)
    compare_lower_stats = wilcoxon(allCCs_mean, CCsL_mean)

    ##for unpaired stats
    # compare_upper_stats = mannwhitneyu(allCCs_mean, CCsU_mean)
    # compare_lower_stats = mannwhitneyu(allCCs_mean, CCsL_mean)

    print("Across vs within:", compare_upper_stats)
    print("Across vs control:", compare_lower_stats)

    #annotate stats
    xmin, xmax = ax_hist.get_xlim()
    markerx = xmax+(xmax-xmin)*0.05
    linex = xmax+(xmax-xmin)*0.15
    textx = xmax+(xmax-xmin)*0.25
    line_kwargs = dict(linewidth = 0.5, color = 'k')
    text_kwargs = dict(ha='left', va='center')

    ax_hist.scatter(markerx, allCCs_median, color = params.colors.MainCC, marker = '<')
    ax_hist.scatter(markerx, CCsU_median, color = params.colors.UpperCC, marker = '<')
    ax_hist.scatter(markerx, CCsL_median, color = params.colors.LowerCC, marker = '<')

    ax_hist.plot([linex, linex], [allCCs_median, CCsU_median], **line_kwargs)
    ax_hist.plot([linex, linex], [allCCs_median, CCsL_median], linestyle = '--', **line_kwargs)
    
    # print(dt.get_signif_annot(compare_upper_stats[1]))
    ax_hist.text(textx, (allCCs_median + CCsU_median)/2, dt.get_signif_annot(compare_upper_stats[1]), **text_kwargs)
    ax_hist.text(textx, (allCCs_median + CCsL_median)/2, dt.get_signif_annot(compare_lower_stats[1]), **text_kwargs)

    

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc()
    fig=plt.figure(dpi=100)
    gs = fig.add_gridspec(nrows=1, ncols=2, width_ratios=(5,1), wspace=0)

    ax = fig.add_subplot(gs[0])
    ax_ = fig.add_subplot(gs[1])

    
    _, allDFs_Str = get_full_mouse_data()
    
    plot_mouse_str_cca(ax,ax_, allDFs_Str)
    
    plt.show()

---

STR decoding

In [ ]:
@utility.report
def plot_str_decoding(ax, AllDFs):
    defs = mouse_defs
    
    #=========================
    within_score = {}
    aligned_score = {}
    unaligned_score = {}
    for i, df1 in enumerate(tqdm(AllDFs)):
        animal1 = df1.mouse[0]

        AllData, AllVel = defs.get_data_array_and_vel([df1], defs.exec_epoch_decode, area=defs.areas[1],
                                                    n_components=defs.n_components)
        # adding history
        AllData = dt.add_history_to_data_array(AllData, defs.MAX_HISTORY)
        AllData = AllData[...,defs.MAX_HISTORY:,:]
        AllVel = AllVel[...,defs.MAX_HISTORY:,:]
        *_,n_time,n_comp = AllData.shape
        AllData1 = AllData[0,...]
        AllVel1 = AllVel[0,...]
        # resizing
        X1 = AllData1.reshape((-1, n_time, n_comp))
        AllVel1 = AllVel1.reshape((-1,n_time,3))

        fold_score =[]
        kf = KFold(n_splits=10)
        for train_index, test_index in kf.split(X1[:,0,0]):
            x_train, x_test = X1[train_index,...], X1[test_index,...]
            y_train, y_test = AllVel1[train_index,...], AllVel1[test_index,...]

            lstm_model = lstm.LSTMDecoder(input_dims=X1.shape[-1], output_dims=3)
            lstm_model.fit(x_train=x_train, y_train=y_train, epochs = 10)
            lstm_model.predict(x_test, y_test)
            fold_score.append(lstm_model.score)
        fold_score = np.median(fold_score)
        within_score[df1.file[0]] = fold_score


        aligned_score[df1.file[0]] = {}
        unaligned_score[df1.file[0]] = {}
        for j, df2 in enumerate(AllDFs):
            if j < i: continue
            animal2 = df2.mouse[0]
            if animal1 == animal2: continue
            
            #================================
            # Unaligned
            AllData, AllVel = defs.get_data_array_and_vel([df1, df2], defs.exec_epoch_decode,
                                                          area=defs.areas[1], n_components=defs.n_components)

            # adding history
            AllData = dt.add_history_to_data_array(AllData, defs.MAX_HISTORY)
            AllData = AllData[...,defs.MAX_HISTORY:,:]
            AllVel = AllVel[...,defs.MAX_HISTORY:,:]
            AllData1 = AllData[0,...]
            AllData2 = AllData[1,...]
            AllVel1 = AllVel[0,...]
            AllVel2 = AllVel[1,...]
            # resizing
            _,n_trial,n_time,n_comp = AllData1.shape
            X1 = AllData1.reshape((-1,n_comp))
            X2 = AllData2.reshape((-1,n_comp))
            AllVel1 = AllVel1.reshape((-1,n_time,3))
            AllVel2 = AllVel2.reshape((-1,n_time,3))

            *_,U,V = dt.canoncorr(X1, X2, fullReturn=True)
            U = U.reshape((-1,n_time,n_comp))
            V = V.reshape((-1,n_time,n_comp))
            X1 = X1.reshape((-1,n_time,n_comp))
            X2 = X2.reshape((-1,n_time,n_comp))

            lstm_model = lstm.LSTMDecoder(input_dims=U.shape[-1], output_dims=3)
            lstm_model.fit(x_train=U, y_train=AllVel1)
            lstm_model.predict(V, AllVel2)
            aligned_score[df1.file[0]][df2.file[0]] = lstm_model.score.mean()

            #================================
            # Unaligned
            lstm_model1 = lstm.LSTMDecoder(input_dims=X1.shape[-1], output_dims=3)
            lstm_model1.fit(x_train=X1, y_train=AllVel1)
            lstm_model1.predict(X2, AllVel2)
            lstm_model2 = lstm.LSTMDecoder(input_dims=X1.shape[-1], output_dims=3)
            lstm_model2.fit(x_train=X2, y_train=AllVel2)
            lstm_model2.predict(X1, AllVel1)

            unaligned_score[df1.file[0]][df2.file[0]] = (lstm_model1.score.mean() + lstm_model2.score.mean()) / 2


    # return within_score, aligned_score, unaligned_score
    #======================== PLOTTING
    pop_within = np.array(list(within_score.values()))
    pop_aligned = np.array([val for key in aligned_score for val in aligned_score[key].values()])
    pop_unaligned = np.array([val for key in unaligned_score for val in unaligned_score[key].values()])

    ax.errorbar(1, pop_aligned.mean(), np.std(pop_aligned), label='Across\n' r'(\textit{aligned})',
                color=params.colors.MainCC, fmt='-o', capsize=1.5)    
    ax.errorbar(0, pop_unaligned.mean(), np.std(pop_unaligned), label='Across\n' r'(\textit{unaligned})',
                color=params.colors.LowerCC, fmt='-o', capsize=1.5)
    ax.errorbar(2, pop_within.mean(), np.std(pop_within), label='Within',
                color=params.colors.UpperCC, fmt='-o', capsize=1.5)

    unal_vals = []
    al_vals = []
    wi_vals = []
    for file1, nested_dict in aligned_score.items():
        wi_val1 = within_score[file1]
        for file2, al_val in nested_dict.items():
            wi_val2 = within_score[file2]
            unal_val = unaligned_score[file1][file2]
            ax.plot([0,1,2], [unal_val, al_val, wi_val1],
                    color='gray', lw=.2, zorder=6, marker = 'o', ms=.1, alpha=.2)
            ax.plot([1,2], [al_val, wi_val2],
                    color='gray', lw=.2, zorder=6, marker = 'o', ms=.1, alpha=.2)

            #for stats
            unal_vals.append(unal_val)
            al_vals.append(al_val)
            wi_vals.append(wi_val1)
            wi_vals.append(wi_val2)
            

    ax.set_xlim([-0.2,2.2])
    ax.set_xticks([0,1,2])
    ax.set_xticklabels(['Across\n' r'(\textit{unaligned})',
                        'Across\n' r'(\textit{aligned})',
                        'Within'])
    ax.set_ylabel('Prediction accuracy ($R^2$)')
    ax.set_ylim([-.05,.8])
    # ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_bounds([0,2])
    ax.spines['left'].set_bounds([0,.8])

    #stats ###########################################################################
    #calc stats
    ##for paired stats
    compare_upper_stats = wilcoxon(np.repeat(al_vals,2), wi_vals)
    compare_lower_stats = wilcoxon(al_vals, unal_vals)

    print("Across vs within:", compare_upper_stats)
    print("Across vs control:", compare_lower_stats)

    #annotate stats
    ymin, ymax = ax.get_ylim()
    liney = ymax*0.95
    texty = ymax*1
    line_kwargs = dict(linewidth = 0.5, color = 'k')
    text_kwargs = dict(ha='center', va='center')

    ax.plot([0,1], [liney, liney], **line_kwargs)
    ax.plot([1,2], [liney, liney], linestyle = '--', **line_kwargs)
    
    ax.text(0.5, texty, dt.get_signif_annot(compare_upper_stats[1]), **text_kwargs)
    ax.text(1.5, texty, dt.get_signif_annot(compare_lower_stats[1]), **text_kwargs)


    return within_score, aligned_score, unaligned_score

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc()
    fig=plt.figure(dpi=100)
    ax = fig.add_subplot()
    
    _, allDFs_Str = get_full_mouse_data()
    
    within_score, aligned_score, unaligned_score = plot_str_decoding(ax, allDFs_Str)
    plt.savefig('dummy.PDF')
    plt.show()

# Part 2
Plot figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc()
    figsize= (params.LargeFig[0], params.panels.SmallH*2+.3)
    fig=plt.figure(figsize=figsize)
    
   

    ##########################################
    # 1: PREP schematics
    gs1   =utility.add_gridspec_abs(fig, nrows=1, ncols=1, top=figsize[1], 
                                    right=params.panels.schmatic[0],
                                    width=params.panels.schmatic[0],
                                    height=params.panels.cca_hist[1])
    ax1   =fig.add_subplot(gs1[0])
    ax1 = utility.phantom_axes(ax1)
    ax1.set_facecolor([1,0,0,0])


    
    
    ##########################################
    # 2: CCA plot for prep
    gs2 = fig.add_gridspec(nrows=1, ncols=2,  width_ratios=(params.panels.cca[0],params.panels.cca_hist[0]-params.panels.cca[0]),
                           left=gs1.right+(.3/figsize[0]),  # .7" offset
                           right=gs1.right+ (.3/figsize[0]) + params.panels.cca_hist[0]/figsize[0],
                           bottom=gs1.bottom,
                           top=gs1.top,
                           wspace=0)

    ax2 = fig.add_subplot(gs2[0])
    ax2_ = fig.add_subplot(gs2[1])
    
    full_list_MCx, allDFs_MCx = get_full_monkey_data()
    
    plot_monkey_cca_prep(ax2,ax2_, full_list_MCx, allDFs_MCx)
    

    
    
    ##########################################
    # 3: Decoding for prep
    gs3   =utility.add_gridspec_abs(fig, nrows=1, ncols=1,
                                    top=figsize[1],
                                    right=figsize[0],
                                    width=params.panels.decoding_hist[0],
                                    height=params.panels.decoding_hist[1])

    ax3 = fig.add_subplot(gs3[0])

    
    plot_monkey_target_decoding(ax3, allDFs_MCx)

        
    
    ##########################################
    # 4: schematics STR
    gs4   =utility.add_gridspec_abs(fig, nrows=1, ncols=1, left=0, 
                                    bottom=gs1.bottom*figsize[1]-params.panels.schmatic[1],
                                    width=params.panels.schmatic[0],
                                    height=params.panels.cca_hist[1])
    ax4   =fig.add_subplot(gs4[0])
    ax4 = utility.phantom_axes(ax4)
    ax4.set_facecolor([0,0,0,0])

    
    
    
    ##########################################
    # 5: CCA plot for STR
    gs5 = fig.add_gridspec(nrows=1, ncols=2,  width_ratios=(params.panels.cca[0],params.panels.cca_hist[0]-params.panels.cca[0]),
                           left=gs2.left,
                           right=gs2.right,
                           bottom=gs4.bottom,
                           top=gs4.top,
                           wspace=0)

    ax5 = fig.add_subplot(gs5[0])
    ax5_ = fig.add_subplot(gs5[1])
    
    
    _, allDFs_Str = get_full_mouse_data()
    
    plot_mouse_str_cca(ax5,ax5_, allDFs_Str)
    

    
    #########################################
    # 6: Decoding for STR
    gs6   =utility.add_gridspec_abs(fig, nrows=1, ncols=1,
                                    top=gs5.top*figsize[1],
                                    right=figsize[0],
                                    width=params.panels.decoding_hist[0],
                                    height=params.panels.decoding_hist[1])

    ax6 = fig.add_subplot(gs6[0])


    within_score, aligned_score, unaligned_score = plot_str_decoding(ax6, allDFs_Str)


    
    
#     fig.align_ylabels([ax1,ax4])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(ax1,ax2,ax3,ax4,ax5,ax6)
    OFFX=np.array([.02]*len(AXES))
    OFFY=np.array([.03]*len(AXES))
    # OFFX[[-1]]=0.12
    # OFFX[[1]]=0.06
    
    params.add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)

    fig.savefig(params.figPath / 'figure3.pdf', format='pdf', bbox_inches='tight')
    
    
    ##########################################
    # monkey schematics
 
    thisPath  =str(params.figPath / 'figure3.pdf')
    sketchPath=str(params.figPath / 'monkey-prep-schematic.pdf')
    if os.path.exists(sketchPath):
        f1=ppdf.PdfFileReader(thisPath).getPage(0)
        f2=ppdf.PdfFileReader(sketchPath).getPage(0)

        f1.mergeTranslatedPage(page2=f2, tx=10, ty=147, expand=False)

        writer=ppdf.PdfFileWriter()
        writer.addPage(f1)
        with open(thisPath,'wb') as f3:
            writer.write(f3)
